* use counter in instruction definition to keep track how number of lines used and supposed location in ROM.mem file
* optional label to help with jumping -> replace with line number at later stage

In [ ]:
import os
from enum import Enum

In [ ]:
# Data dir
data_dir = '../data'

In [ ]:
def bin_format(num:int):
    return f'{num:02X}'

In [ ]:
# ROM and RAM size
ROM_SIZE = 2**8
RAM_SIZE = 2**7

# Base addresses
MOUSE_BASE_ADDR   = int('0xA0', base=16)
LEDS_BASE_ADDR    = int('0xC0', base=16)
SEG7_BASE_ADDR    = int('0xD0', base=16)
SWITCH_BASE_ADDR  = int('0xE0', base=16)

# ALU op-codes
ALU_OPS = Enum('ALU_OPS',
               names=['ADD', 'SUB', 'MUL', 'SL_A',
                      'SR_A', 'INC_A', 'INC_B', 'DEC_A',
                      'DEC_B', 'EQ', 'GT', 'LT',
                      'OUT_A'],
               start=0,
               type=int)

ALU_OPS_COMMENTS = {
    ALU_OPS.ADD : 'A + B',
    ALU_OPS.SUB : 'A - B', 
    ALU_OPS.MUL : 'A * B', 
    ALU_OPS.SL_A : 'A << 1', 
    ALU_OPS.SR_A : 'A >> 1', 
    ALU_OPS.INC_A : 'A + 1', 
    ALU_OPS.INC_B : 'B + 1', 
    ALU_OPS.DEC_A : 'A - 1', 
    ALU_OPS.DEC_B : 'B - 1', 
    ALU_OPS.EQ: 'A == B', 
    ALU_OPS.GT : 'A > B', 
    ALU_OPS.LT : 'A < B', 
    ALU_OPS.OUT_A : 'A'
}

# Conditional branch types
BRANCH_TYPES = Enum('BRANCH_TYPES',
                    names=['EQ', 'GT', 'LT'],
                    start=9,
                    type=int)

# Instruction codes
READ_MEM_TO_A  = int('0000', base=2)
READ_MEM_TO_B  = int('0001', base=2)
WRITE_A_TO_MEM = int('0010', base=2)
WRITE_B_TO_MEM = int('0011', base=2)
ALU_OP_TO_A    = int('0100', base=2)
ALU_OP_TO_B    = int('0101', base=2)
BRANCH         = int('0110', base=2)
GOTO           = int('0111', base=2)
GOTO_IDLE      = int('1000', base=2)
FUNC_CALL      = int('1001', base=2)
RETURN         = int('1010', base=2)
DEREF_A        = int('1011', base=2)
DEREF_B        = int('1100', base=2)

def read_mem_to_A(mem_addr:int, ram_size=RAM_SIZE, check=True):
    if check:
        assert mem_addr >= 0 and mem_addr < ram_size,\
            f'Memory address {mem_addr} out of range for {ram_size} bytes RAM!'
    
    comment = f' // A <- Mem[{mem_addr}]'
    
    return '\n'.join([bin_format(READ_MEM_TO_A) + comment, bin_format(mem_addr)])

def read_mem_to_B(mem_addr:int, ram_size=RAM_SIZE, check=True):
    if check:
        assert mem_addr >= 0 and mem_addr < ram_size,\
            f'Memory address {mem_addr} out of range for {ram_size} bytes RAM!'
    
    comment = f' // B <- Mem[{mem_addr}]'
    
    return '\n'.join([bin_format(READ_MEM_TO_B) + comment, bin_format(mem_addr)])

def write_A_to_mem(mem_addr:int, ram_size=RAM_SIZE, check=True):
    if check:
        assert mem_addr >= 0 and mem_addr < ram_size,\
            f'Memory address {mem_addr} out of range for {ram_size} bytes RAM!'
    
    comment = f' // Mem[{mem_addr}] <- A'
    
    return '\n'.join([bin_format(WRITE_A_TO_MEM) + comment, bin_format(mem_addr)])

def write_B_to_mem(mem_addr:int, ram_size=RAM_SIZE, check=True):
    if check:
        assert mem_addr >= 0 and mem_addr < ram_size,\
            f'Memory address {mem_addr} out of range for {ram_size} bytes RAM!'
    
    comment = f' // Mem[{mem_addr}] <- B'
    
    return '\n'.join([bin_format(WRITE_B_TO_MEM) + comment, bin_format(mem_addr)])

def alu_to_A(op_code:int):
    assert op_code in ALU_OPS._value2member_map_, f'Op-code {op_code} is not valid!'
    
    comment = f' // A <- {ALU_OPS_COMMENTS[op_code]}'
    
    return bin_format((op_code << 4) + ALU_OP_TO_A) + comment

def alu_to_B(op_code:int):
    assert op_code in ALU_OPS._value2member_map_, f'Op-code {op_code} is not valid!'
    
    comment = f' // B <- {ALU_OPS_COMMENTS[op_code]}'
    
    return bin_format((op_code << 4) + ALU_OP_TO_B) + comment

def breq(mem_addr:int, label=None, rom_size=ROM_SIZE):
    assert mem_addr >= 0 and mem_addr < rom_size,\
        f'Memory address {mem_addr} out of range for {rom_size} bytes ROM!'
    
    label = label + ' at ' if label else ''
    
    comment = f' // if A == B go to {label}ROM[{mem_addr}]'
    
    return '\n'.join([bin_format((BRANCH_TYPES.EQ << 4) + BRANCH) + comment, bin_format(mem_addr)])

def bgtq(mem_addr:int, label=None, rom_size=ROM_SIZE):
    assert mem_addr >= 0 and mem_addr < rom_size,\
        f'Memory address {mem_addr} out of range for {rom_size} bytes ROM!'
    
    label = label + ' at ' if label else ''
    
    comment = f' // if A > B go to {label}ROM[{mem_addr}]'
    
    return '\n'.join([bin_format((BRANCH_TYPES.GT << 4) + BRANCH) + comment, bin_format(mem_addr)])

def bltq(mem_addr:int, label=None, rom_size=ROM_SIZE):
    assert mem_addr >= 0 and mem_addr < rom_size,\
        f'Memory address {mem_addr} out of range for {rom_size} bytes ROM!'
    
    label = label + ' at ' if label else ''
    
    comment = f' // if A < B go to {label}ROM[{mem_addr}]'
    
    return '\n'.join([bin_format((BRANCH_TYPES.LT << 4) + BRANCH) + comment, bin_format(mem_addr)])

def goto(mem_addr:int, label=None, rom_size=ROM_SIZE):
    assert mem_addr >= 0 and mem_addr < rom_size,\
        f'Memory address {mem_addr} out of range for {rom_size} bytes ROM!'
    
    label = label + ' at ' if label else ''
    
    comment = f' // Go to {label}ROM[{mem_addr}]'
    
    return '\n'.join([bin_format(GOTO) + comment, bin_format(mem_addr)])

def goto_idle():
    comment = ' // Go to Idle state and wait for Interrupts'
    return bin_format(GOTO_IDLE) + comment

def func_call(mem_addr:int, rom_size=ROM_SIZE):
    assert mem_addr >= 0 and mem_addr < rom_size,\
        f'Memory address {mem_addr} out of range for {rom_size} bytes ROM!'
    
    comment = f' // Function call to ROM[{mem_addr}]. Context saved.'
    
    return '\n'.join([bin_format(FUNC_CALL) + comment, bin_format(mem_addr)])

def func_return():
    comment = ' // Restoring saved context after function call.'
    return bin_format(RETURN) + comment

def deref_A():
    comment = ' // A <- Mem[A]'
    return bin_format(DEREF_A) + comment

def deref_B():
    comment = ' // B <- Mem[B]'
    return bin_format(DEREF_B) + comment

In [ ]:
def insert_functions(program:list, functions:list, rom_size:int=ROM_SIZE):
    prog_copy = [''] * rom_size
    
    for i, line in enumerate('\n'.join(program).split('\n')):
        prog_copy[i] = line 
    
    for function, idx in functions:
        # Check function fits in the ROM
        assert idx + len(function) <= rom_size,\
            f'Function length of {len(function)} is too long at index={idx} from {rom_size} bytes ROM.'

        # Insert the function line by line
        for line in '\n'.join(function).split('\n'):
            # Check location in ROM is empty
            if prog_copy[idx]:
                raise Exception(f'ROM region {idx} is not empty. Risk of overwriting instructions.')

            # Insert function line and move ROM index
            prog_copy[idx] = line
            idx += 1
    
    return prog_copy

def generate_rom(program, filename, size=2**8):
    rom = '\n'.join(program)
    rom_size = len(rom.split('\n'))
    assert rom_size <= size, f'Program is {rom_size} bytes for {size} bytes ROM.'
    
    # Fill empty lines in ROM with FF
    for i, line in enumerate(program):
        if not line: program[i] = 'FF'
    
    rom = '\n'.join(program)
    
    # # Fill rest of ROM with 00
    # if rom_size < size:
    #     rom = rom + '\n' + '\n'.join(['FF']*(size-rom_size))
    
    with open(filename, 'w') as f:
        f.write(rom)
        
def generate_ram(data_entries, filename, size=2**7):
    ram = [0]*size
    for i, d in data_entries:
        ram[i] = d
    
    ram = '\n'.join(map(bin_format, ram))
    with open(filename, 'w') as f:
        f.write(ram)

### ROM data generation

In [ ]:
ROM_ADDR_WIDTH = 8
ADDR_COUNT = 2 ** ROM_ADDR_WIDTH
ROM_FILE = os.path.join(data_dir, 'ROM_test.mem')
data = '\n'.join([f'{i:02X}' for i in range(ADDR_COUNT)])
with open(ROM_FILE, 'w') as f:
    f.write(data)

### RAM test data generation

In [ ]:
RAM_ADDR_WIDTH = 7
ADDR_COUNT = 2 ** RAM_ADDR_WIDTH
RAM_FILE = os.path.join(data_dir, 'RAM_test.mem')
data = '\n'.join([f'{i:02X}' for i in range(ADDR_COUNT)])
with open(RAM_FILE, 'w') as f:
    f.write(data)

### Processor TB ROM and RAM generation

In [ ]:
program = [
    # Memory instruction testing
    read_mem_to_A(0),
    read_mem_to_B(1),
    write_A_to_mem(2),
    write_B_to_mem(3),
    read_mem_to_A(3),
    read_mem_to_B(2), # line 12
    
    # ALU operation testing
    alu_to_A(ALU_OPS.ADD),
    alu_to_B(ALU_OPS.ADD),
    alu_to_B(ALU_OPS.SUB),
    alu_to_A(ALU_OPS.SUB),
    alu_to_A(ALU_OPS.MUL),
    alu_to_B(ALU_OPS.MUL),
    alu_to_A(ALU_OPS.SL_A),
    alu_to_A(ALU_OPS.SR_A),
    alu_to_B(ALU_OPS.SL_A),
    alu_to_B(ALU_OPS.SR_A),
    alu_to_A(ALU_OPS.INC_A),
    alu_to_A(ALU_OPS.DEC_A),
    alu_to_B(ALU_OPS.INC_B),
    alu_to_B(ALU_OPS.DEC_B),
    alu_to_A(ALU_OPS.EQ),
    alu_to_B(ALU_OPS.EQ),
    alu_to_A(ALU_OPS.EQ),
    alu_to_A(ALU_OPS.GT),
    alu_to_A(ALU_OPS.LT),
    alu_to_B(ALU_OPS.EQ),
    alu_to_B(ALU_OPS.LT),
    alu_to_B(ALU_OPS.GT),
    alu_to_B(ALU_OPS.INC_B),
    alu_to_B(ALU_OPS.OUT_A),
    alu_to_A(ALU_OPS.OUT_A), # line 37
    
    # Conditional branch testing
    breq(41), # Go to line 42 (address 41)
    read_mem_to_A(0), # Skipped
    bltq(45), # Go to line 46 (address 45)
    read_mem_to_A(0),
    bgtq(49), # Go to line 50 (address 49)
    read_mem_to_A(1), # Skipped
    read_mem_to_B(1), # Line 50
    
    # Unconditional branch
    goto(54), # Go to line 55 (address 54)
    alu_to_A(ALU_OPS.SL_A), # Line 54
    
    # Go to Idle State and wait for Interrupts
    goto_idle(), # Line 55
    
    # Functional call testing
    func_call(253), # Line 56
    
    # Register dereference testing
    deref_A(),
    deref_B()
]

# Insert mouse handling function
functions = [
    ([bin_format(55), bin_format(55)], 254),
    ([func_return()], 253)  
]

program = insert_functions(program, functions)

rom_path = os.path.join(data_dir, 'ROM_processor_test.mem')
generate_rom(program, rom_path)

In [ ]:
data_entries = [
    (0, 42),
    (1, 69),
    (42, 69),
    (69, 42)
]

ram_path = os.path.join(data_dir, 'RAM_processor_test.mem')
generate_ram(data_entries, ram_path)

### Mouse Demo code generation -> WORKS!!!

In [ ]:
demo_rom_path = os.path.join(data_dir, 'ROM_MouseDemo.mem')
demo_ram_path = os.path.join(data_dir, 'RAM_MouseDemo.mem')

In [ ]:
mouse_program = [
    ############################
    # Start of Mouse interrupt
    
    # Read Mouse Status
    read_mem_to_A(MOUSE_BASE_ADDR, check=False),
    # Set dataline with Mouse Status for Seg7 to read
    write_A_to_mem(MOUSE_BASE_ADDR, check=False),
    
    # Read Mouse X and stored it back to RAM
    read_mem_to_A(MOUSE_BASE_ADDR + 1, check=False),
    write_A_to_mem(MOUSE_BASE_ADDR + 1, check=False),
    write_A_to_mem(LEDS_BASE_ADDR + 1, check=False), # Set upper 8 LEDs with X
    
    # Read Mouse Y and stored it back to RAM
    read_mem_to_A(MOUSE_BASE_ADDR + 2, check=False),
    write_A_to_mem(MOUSE_BASE_ADDR + 2, check=False),
    write_A_to_mem(LEDS_BASE_ADDR, check=False), # Set lower 8 LEDs with Y
    
    # At the end of the service, go back to IDLE and
    # wait for the next interrupt
    goto_idle(),

    # End of Mouse interrupt
    ############################
    # Start of Timer interrupt
    
    # Update slide switch
    read_mem_to_A(SWITCH_BASE_ADDR, check=False),
    # write_A_to_mem(SWITCH_BASE_ADDR, check=False),
    write_A_to_mem(SEG7_BASE_ADDR, check=False),
    
    # At the end of the service, go back to IDLE and
    # wait for the next interrupt
    goto_idle()
    
    # End of Timer interrupt
    ############################
]

mouse_functions = [
    ([bin_format(0)], 255), # Mouse Interrupt servicing start address is 0x00
    ([bin_format(17)], 254), # Timer Interrupt servicing start address is 0x11 (17)
    
]

mouse_program = insert_functions(mouse_program, mouse_functions)

mouse_data_entries = []

In [ ]:
generate_rom(mouse_program, demo_rom_path)
generate_ram(mouse_data_entries, demo_ram_path)

### Mouse Demo fancy seg7 protocol -> doesn't work

In [ ]:
demo_rom_path2 = os.path.join(data_dir, 'ROM_MouseDemo2.mem')
demo_ram_path2 = os.path.join(data_dir, 'RAM_MouseDemo2.mem')

In [ ]:
mouse_program2 = [
    ############################
    # Start of Mouse interrupt
    
    # Read Mouse Status and save it to RAM
    read_mem_to_A(MOUSE_BASE_ADDR, check=False),
    write_A_to_mem(0, check=False), # Save it in RAM: Mem[0x00] <- MouseStatus
    
    # Read Mouse X and stored it to RAM
    read_mem_to_A(MOUSE_BASE_ADDR + 1, check=False),
    write_A_to_mem(1, check=False), # Save it in RAM: Mem[0x01] <- MouseX
    write_A_to_mem(LEDS_BASE_ADDR + 1, check=False), # Set upper 8 LEDs with X
    
    # Read Mouse Y and stored it to RAM
    read_mem_to_A(MOUSE_BASE_ADDR + 2, check=False),
    write_A_to_mem(2, check=False), # Save it in RAM: Mem[0x02] <- MouseY
    write_A_to_mem(LEDS_BASE_ADDR, check=False), # Set lower 8 LEDs with Y
    
    # Update slide switch
    read_mem_to_A(SWITCH_BASE_ADDR, check=False), # A <- DisplayOpt
    alu_to_B(ALU_OPS.OUT_A), # Copy A to B: B <- A
    deref_B(), # Load value stored at address: either MouseStatus, MouseX or MouseY
    write_A_to_mem(SEG7_BASE_ADDR, check=False), # Send DisplayOpt to seg7
    write_B_to_mem(SEG7_BASE_ADDR + 1, check=False), # Send DataPacket to seg7
    
    # At the end of the service, go back to IDLE and
    # wait for the next interrupt
    goto_idle(),

    # End of Mouse interrupt
    ############################
    # Start of Timer interrupt
    
    # # Update slide switch
    # read_mem_to_A(SWITCH_BASE_ADDR, check=False), # A <- DisplayOpt
    # alu_to_B(ALU_OPS.OUT_A), # Copy A to B: B <- A
    # deref_B(), # Load value stored at address: either MouseStatus, MouseX or MouseY
    # write_A_to_mem(SEG7_BASE_ADDR, check=False), # Send DisplayOpt to seg7
    # write_B_to_mem(SEG7_BASE_ADDR + 1, check=False), # Send DataPacket to seg7
    
    # At the end of the service, go back to IDLE and
    # wait for the next interrupt
    goto_idle()
    
    # End of Timer interrupt
    ############################
]

mouse_functions2 = [
    ([bin_format(0)], 255), # Mouse Interrupt servicing start address is 0x00
    ([bin_format(24)], 254), # Timer Interrupt servicing start address is 0x11 (17)
    
]

mouse_program2 = insert_functions(mouse_program2, mouse_functions2)

mouse_data_entries2 = []

generate_rom(mouse_program2, demo_rom_path2)
generate_ram(mouse_data_entries2, demo_ram_path2)